# Extractions données d'historique Youtube

Le workflow suivant s'appuie sur les historiques de recherche et de vidéos visonnées. D'audtres données sont égallement disponibles mais non traité ici.
Cette annalyse simple de données vise à montrer qu'il est facile d'extraire des informations et d'en apprendre plus sur les données de l'utilisateur.

Les données qui nous intéressent sont dans le dossier YouTube et YouTube Music/historique. Deux fichiers se trouvent dans ce dossier:
    * "Historique des recherches" : informations sur les recherches effectuées
    * "watch-history": informations sur les vidéos visionnées
Ces deux fichiers peuvent être télécharger au format html(défault) ou json. Pour faciliter l'extraction nous allons utiliser les fichiers json. 

Attention: les chemins et noms de fichiers vont varier en fonctions de où votre notebook se situe et de la langue de téléchargement (changement des nomns de dossier et fichiers).
Ici les chemins sont pour des noms de dossiers en français avec un notebook situé au même niveau que le dossier Takeout.

Des actions de la part de l'utilisateurs sont nécéssaire que dans les deux premières parties (import et chemins).
Ensuite les résultats d'analyse seront directement visibles dans la partie résultats.

## Imports et installations nécéssaires

Ci dessous se trouve tout les imports nécéssaires. Des installation peuvent être nécéssaires, vous trouverez en commentaire la ligne de commande pour réaliser l'installation.

In [ ]:
import pandas as pd
import os
import tldextract #pip install tldextract
import IPython
import matplotlib.pyplot as plt
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import nltk # pip install nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2


from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.ensemble import AdaBoostClassifier

#pip install keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical

from wordcloud import WordCloud
import numpy as np
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## Chemins
Ci-dessous remplacer les chemins par ceux correspondants pour vous. Si le fichier est bien chargé et au bon format un message le confirmant s'affichera, dans le cas contraire un message d'erreur s'affichera. 
Cela est la dernière étape où vous avez à intervenir. 

In [ ]:
# chemin fichier "Historique des recherches"
json_Hrecherche='Takeout_YT_json/YouTube et YouTube Music/historique/Historique des recherches.json'

# chemin fichier "Historique des vidéos regardés" (watch)
json_Hwatch='Takeout_YT_json/YouTube et YouTube Music/historique/watch-history.json'


# Vérification: existance et format
print("\n") 

    #recherche
try:
 with open(json_Hrecherche): pass
 extension=os.path.splitext(json_Hrecherche)
 if ".json" in extension[-1]:
    print("Le fichier d'historique des recherches au format json est bien chargé.\n") 
 else:
    print("Erreur! Le fichier d'historique des recherches spécifié n'est pas au bon format. Le format attendu est json.\n")
except IOError:
 print("Erreur! Le chemin indiqué pour le fichier d'historique des recherches spécificié n'est pas bon, le fichier est introuvable (cela peut venir du format de votre fichier).\n")

    #watch
try:
 with open(json_Hwatch): pass
 extension=os.path.splitext(json_Hwatch)
 if ".json" in extension[-1]:
    print("Le fichier d'historique des vidéos regardées au format json est bien chargé.\n") 
 else:
    print("Erreur! Le fichier d'historique des vidéos regardées spécifié n'est pas au bon format. Le format attendu est json.\n")
except IOError:
 print("Erreur! Le chemin indiqué pour le fichier d'historique des vidéos regardées spécificié n'est pas bon, le fichier est introuvable (cela peut venir du format de votre fichier).")

## Code analyse du fichier "Historique des recherches"

In [ ]:
# création du data frame

df_Hrecherche = pd.read_json(json_Hrecherche)

# élimination du texte redondant ajouté devant les titres
df_Hrecherche['title'] = df_Hrecherche['title'].str.replace(r'Vous avez recherché', '') 
# sauvegarde des recherches completes (car modifications ensuites)
df_Hrecherche['title_comp']= df_Hrecherche['title']

In [ ]:
##
#Nettoyage des données "title", correspondent aux recherches effectuées
##
# conversion en minuscule
df_Hrecherche['title'] = df_Hrecherche['title'].map(lambda x: x.lower())

# retrait de la ponctuation (caractères spéciaux tels que $, ! ...)
df_Hrecherche['title'] = df_Hrecherche['title'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))

# retrait des espaces vides inutiles 
df_Hrecherche['title'] = df_Hrecherche['title'].map(lambda x: x.strip())

# conversion de la str en liste de "tocken" (nécessaire pour l'étape suivante)
df_Hrecherche['title'] = df_Hrecherche['title'].map(lambda x: word_tokenize(x))
 
# élimine les mots non-alphabétiques
df_Hrecherche['title'] = df_Hrecherche['title'].map(lambda x: [word for word in x if word.isalpha()])

# filtre les "stop words", mots considérés comme sans apport d'infomation (tel que and/et the/le-la ...)
stop_words = set(stopwords.words("english"))
stop_words2 =set(stopwords.words("french"))
df_Hrecherche['title'] = df_Hrecherche['title'].map(lambda x: [w for w in x if not w in stop_words])
df_Hrecherche['title'] = df_Hrecherche['title'].map(lambda x: [w for w in x if not w in stop_words2])

# retransforme les liste en str
df_Hrecherche['title'] = df_Hrecherche['title'].map(lambda x: ' '.join(x))


# supprimer les lignes avec title vides
df_Hrecherche.drop(df_Hrecherche.loc[df_Hrecherche['title']==''].index, inplace=True)

In [ ]:
df_Hrecherche.head(2)

In [ ]:
# Extraction des dates au bon format pour des comparaison à suivre, année-mois-jour 

df_Hrecherche["year"], df_Hrecherche["autre2"]= df_Hrecherche["time"].str.split("-", 1 ).str # pour récupérer l'année
df_Hrecherche["date"], df_Hrecherche["autre"] = df_Hrecherche["time"].str.split("T", 1).str
df_Hrecherche["date"]=pd.to_datetime(df_Hrecherche["date"], errors = 'coerce')
# Elimination des collones innutiles (nettoyage)
df_Hrecherche=df_Hrecherche[['title','title_comp','year','date']]

In [ ]:
# Observation des valeurs manquantes dans les recherches effectuées, sont normalement à 0
num_missing_desc = df_Hrecherche.isnull().sum()[2] 
print('Nombre de valeurs manquantes: ' + str(num_missing_desc))

In [ ]:
# transformations des valeurs années (year) en nombre car considérés comme str (due à la méthode de récupération)
df_Hrecherche[['year']] = df_Hrecherche[['year']].apply(pd.to_numeric) 
#df_Hrecherche.dtypes # verification des types de chaque colonnes 

In [ ]:
#ajout des jours de la semaine en fonction de la date
df_Hrecherche["jour"] = df_Hrecherche["date"].dt.day_name()

In [ ]:
# reset de l'index, cette étape est crutiale pour la suite et le bon affichage des résultats
# le formatage et l'élimination de certaines valeurs induisent des biais dans l'index et il est important de le réinitialiser
df_Hrecherche=df_Hrecherche.reset_index()

# Elimination de la collone index crée qui n'est pas utile
df_Hrecherche=df_Hrecherche.drop(columns=["index"])

In [ ]:
# récupération des informations sur la première et dernière recherche
dateHF= df_Hrecherche.iloc[-1,3] 
dateHF=dateHF.date()
titleHF= df_Hrecherche.iloc[-1,1] 
dateHL= df_Hrecherche.iloc[1,3] 
dateHL=dateHL.date()
titleHL= df_Hrecherche.iloc[1,1] 

In [ ]:
# réalistation d'un regroupement par année et jours de la semaine
df_Hrecherche['nb'] = 1 # ajout d'un poid de 1 à chaque ligne
summary_recherches = pd.DataFrame(df_Hrecherche.groupby(['year',"jour"]).count().astype('int')["nb"]) 
summary_recherches = summary_recherches.unstack(level=0)
summary_recherches = summary_recherches.fillna(0)


# afficher les jours de la semaine dans le bon ordre
jour = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
summary_recherches = summary_recherches.reindex(jour)

In [ ]:
#fig1=summary_recherches.plot(kind='line', subplots=False)
#fig1.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
#fig1.set_title("Nombre de recherche effectuées en fonction \n des jours de la semaine et des années")

In [ ]:
#Représentation du nombre de recherche en fonction des années
summary_recherchesA = pd.DataFrame(df_Hrecherche.groupby(['year']).count().astype('int')["nb"]) 

#fig2=summary_recherchesA.plot(kind='bar', subplots=False).set_title("Nombre de recherche effectuées en fonction des années")

In [ ]:
#Pour représenter en fonction des jours seulement (années mergées)
summary_recherchesJ = pd.DataFrame(df_Hrecherche.groupby(['jour']).count().astype('int')["nb"]) 
# afficher les jours de la semaine dans le bon ordre
jour = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
summary_recherchesJ = summary_recherchesJ.reindex(jour)

#représentation graphique
#fig3=summary_recherchesJ.plot(kind='line', subplots=False).set_title("Nombre de recherche effectuées en fonction des jours de la semaine")
#fig3=summary_recherchesJ.plot(kind='bar', subplots=False).set_title("Nombre de recherche effectuées en fonction des jours de la semaine")

In [ ]:
# ajout d'une colone avec les fréquences
df_Hrecherche['freq']=df_Hrecherche.groupby(by='title')['title'].transform('count')

In [ ]:
#par CHAINE DE CARACTERES
# création avec les recherches et leurs count
df_countR= df_Hrecherche['title'].value_counts().to_frame()
df_countR=df_countR.reset_index()
df_countR=df_countR.rename(columns = {'index': 'recherche' , 'title':'count'})
df_countR=df_countR[~df_countR.recherche.str.contains("www")]
df_countR=df_countR.reset_index()
df_countR=df_countR.drop(columns=["index"])                        

In [ ]:
# récupération des 5 recherches es plus effectuées
XrM=df_countR['recherche'].head( n=5)
YrM=df_countR['count'].head( n=5)

In [ ]:
#par MOTS
# séparation des recherches par mots et compte la fréquences de ces derniers
wordC2 = df_Hrecherche.title.str.split(expand=True).stack().value_counts().to_frame()
wordC2 =wordC2.reset_index()
wordC2.columns = ["word",'count']

In [ ]:
# récupération des infomation des 5 mots les plus utilisés 
XrW=wordC2['word'].head( n=5)
YrW=wordC2['count'].head( n=5)

In [ ]:
# récupération des mots dans une liste
textR = df_Hrecherche.title.str.split().tolist()
textR = [val for sublist in textR for val in sublist]
textR = str(textR)

In [ ]:
# répupération du masque pour donner la forme au wordcloud
yt_mask = np.array(Image.open( "YT.jpg"))

In [ ]:
## création du wordcloud
#wordcloud = WordCloud(max_words=200 ,mask=yt_mask, background_color="white" , color_func=lambda *args, **kwargs: "red").generate(textR)
#plt.figure()
#plt.imshow(wordcloud, interpolation="bilinear")
#plt.axis("off")
#plt.margins(x=0, y=0)
#plt.show()


## Code analyse du fichier "watch_history" (historique des vidéos regardées)

In [ ]:
# création du data frame

df_Hwatch = pd.read_json(json_Hwatch)
# élimination du texte redondant ajouté devant les title
df_Hwatch['title'] = df_Hwatch['title'].str.replace(r'Vous avez regardé ', '')
# colonne pour garder les titres sans modification de coté
df_Hwatch['title_comp']= df_Hwatch['title']

In [ ]:
##
#Nettoyage des données "title", correspondent aux recherches effectuées
##
# converstion en minuscule
df_Hwatch['title'] = df_Hwatch['title'].map(lambda x: x.lower())

# retrait de la ponctuation (charactères spéciaux tels que $, ! ...)
df_Hwatch['title'] = df_Hwatch['title'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))

# retrait des espaces vides inutiles 
df_Hwatch['title'] = df_Hwatch['title'].map(lambda x: x.strip())

# conversion de la str en liste de "tocken" (nécessaire pour l'étape suivante)
df_Hwatch['title'] = df_Hwatch['title'].map(lambda x: word_tokenize(x))
 
# élimine les mots non-alphabétiques
df_Hwatch['title'] = df_Hwatch['title'].map(lambda x: [word for word in x if word.isalpha()])

# filtre les "stop words", mots considérés comme sans apport d'infomation (tel que and/et the/le-la ...)
stop_words = set(stopwords.words("english"))
stop_words2 =set(stopwords.words("french"))
df_Hwatch['title'] = df_Hwatch['title'].map(lambda x: [w for w in x if not w in stop_words])
df_Hwatch['title'] = df_Hwatch['title'].map(lambda x: [w for w in x if not w in stop_words2])
# retransforme la liste en str
df_Hwatch['title'] = df_Hwatch['title'].map(lambda x: ' '.join(x))


# supprimer les lignes avec title vides
df_Hwatch.drop(df_Hwatch.loc[df_Hwatch['title']==''].index, inplace=True)

In [ ]:
# Extraction des dates au bon format pour des comparaison à suivre, année-mois-jour 
# autre méthode que précédement mais même résultat
df_Hwatch["year"], df_Hwatch["autre2"]= df_Hwatch["time"].str.split("-", 1 ).str # pour récupérer l'année

df_Hwatch["year"], df_Hwatch["month"], df_Hwatch["autre"] = df_Hwatch["time"].str.split("-", 2).str
df_Hwatch["day"], df_Hwatch["autre2"] = df_Hwatch["autre"].str.split("T", 2).str
df_Hwatch['date'] = pd.to_datetime(df_Hwatch[['year', 'month', 'day']], errors = 'coerce')

# Elimination des collones innutiles (nettoyage)
df_Hwatch=df_Hwatch[["title","titleUrl","title_comp","year","date"]]

In [ ]:
# Observation des valeurs manquantes dans les recherches effectuées, sont normalement à 0
num_missing_desc = df_Hwatch.isnull().sum()[0]    # nombre de valeurs dont la recherche est manquante (title)
print('Nombre de valeurs manquantes: ' + str(num_missing_desc))

In [ ]:
# transformations des valeurs années (year) en nombre car considérés comme str (due à la méthode de récupération)
df_Hwatch[['year']] = df_Hwatch[['year']].apply(pd.to_numeric) 
#df_Hwatch.dtypes # la date est bien au bon format

In [ ]:
#ajout des jours de la semaine
df_Hwatch["jour"] = df_Hwatch["date"].dt.day_name()

In [ ]:
#suppréssion des lignes avec vidéos supprimées
df_Hwatch=df_Hwatch[~df_Hwatch.title.str.contains("vidéo supprimée")]

In [ ]:
# reset de l'index
df_Hwatch=df_Hwatch.reset_index() # pour etre sure de l'ordre de l'index
df_Hwatch=df_Hwatch.drop(columns=["index"])

In [ ]:
# récupération d'informationss sur le première et dernière vidéo regardée
dateHwF= df_Hwatch.iloc[-1,4] 
dateHwF=dateHwF.date()
titleHwF= df_Hwatch.iloc[-1,2] 
lienHwF=df_Hwatch.iloc[-1,1]

dateHwL= df_Hwatch.iloc[1,4] 
dateHwL=dateHwL.date()
titleHwL= df_Hwatch.iloc[1,2] 
lienHwL=df_Hwatch.iloc[1,1] 

In [ ]:
# réalistation d'un regroupement par année et jours de la semaine
df_Hwatch['nb'] = 1
summary_watch = pd.DataFrame(df_Hwatch.groupby(['year',"jour"]).count().astype('int')["nb"]) 

In [ ]:
summary_watch = summary_watch.unstack(level=0)
summary_watch = summary_watch.fillna(0)

Le volume de données étant beaucoup plus important on a moins d'historique de vidéo reagrdé que de vidéo recherchées.

In [ ]:
# afficher les jours de la semaine dans le bon ordre
jour = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
summary_watch = summary_watch.reindex(jour)

In [ ]:
#Représentation du nombre de recherche en fonction des années et des jours de la semaine
#fig5=summary_watch.plot(kind='line', subplots=False)
#fig5.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
#fig5.set_title("Nombre de vidéos regardées en fonction \n des années et des jours de la semaine")

In [ ]:
#A l'échelle des année
summary_watchA = pd.DataFrame(df_Hwatch.groupby(['year']).count().astype('int')["nb"]) 

#fig6=summary_watchA.plot(kind='bar', subplots=False).set_title("Nombre de vidéos regardées en fonction des années")

In [ ]:
#A l'échelle des jours 
summary_watchJ = pd.DataFrame(df_Hwatch.groupby(['jour']).count().astype('int')["nb"]) 
# afficher les jours de la semaine dans le bon ordre
jour = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
summary_watchJ = summary_watchJ.reindex(jour)

#fig7=summary_watchJ.plot(kind='bar', subplots=False).set_title("Nombre de vidéos regardées en fonction des jours de la semaine")

In [ ]:
# ajout du nombre de fois que chaque vidéo a été regardée 
df_Hwatch['freq']=df_Hwatch.groupby(by='title')['title'].transform('count')

df_countW= df_Hwatch['title_comp'].value_counts().to_frame()
df_countW=df_countW.reset_index()
df_countW.columns = ["titre",'count']
df_countW=df_countW[~df_countW.titre.str.contains("www")] # suppression des vidéo dont on a pas le titre

In [ ]:
#Récupération des information des vidéo les plus regardées
Xw=df_countW['titre'].head( n=5)
Yw=df_countW['count'].head( n=5)

# Résultats

Nos historiques peuvent en apprendre  beaucoup sur nous, cette petite analyse de vos historiques youtubes ne représente qu'une fraction de ce qu'il est possible d'en extraire.
Pensez-vous connaitre vos habitudes?

### Est-ce que vous vous rappelez de votre première* et de votre dernière recherche?

* première réfère ici à la première recherche référencée dans le fichier, elle peut différer de votre actuelle première recherche

In [ ]:
print('Votre première recherche date du: {0}, et était: \"{1}\". \n'.format(dateHF,titleHF))
print('Votre dernière recherche date du: {0}, et était: \"{1}\".'.format(dateHL,titleHL))

### Avez vous une idée de l'évolution du nombre de vos recherche?

In [ ]:
fig2=summary_recherchesA.plot(kind='bar', subplots=False).set_title("Nombre de recherche effectuées en fonction des années")

### Et de la répartition de ces dernières au cours d'une semaine?

In [ ]:
fig1=summary_recherches.plot(kind='line', subplots=False)
fig1.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
fig1.set_title("Nombre de recherche effectuées en fonction \n des jours de la semaine et des années")
fig3=summary_recherchesJ.plot(kind='line', subplots=False).set_title("Nombre de recherche effectuées en fonction des jours de la semaine")


Les figures ci dessus peuvent en dire beaucoup sur vous et vos habitudes!

### Avez-vous une idée des recherches que vous avez éffectuées le plus?

In [ ]:
print('Vos recherches les plus fréquentes sont \"{0[0]}\": {1[0]} fois, \"{0[1]}\": {1[1]} fois, \"{0[2]}\": {1[2]} fois, \"{0[3]}\": {1[3]} fois, \"{0[4]}\": {1[4]} fois. '.format(XrM,YrM))
print('\nEt les mots que vous avez le plus utilisé dans vos recherches sont \"{0[0]}\": {1[0]} fois, \"{0[1]}\": {1[1]} fois, \"{0[2]}\": {1[2]} fois, \"{0[3]}\": {1[3]} fois, \"{0[4]}\": {1[4]} fois. '.format(XrW,YrW))


Est ce que vous vous y attendiez?

### La figure ci dessous donne un apperçu plus global de vos recherches, seriez vous a l'aise si tout le monde pouvais y avoir accès?

In [ ]:
# création du wordcloud
wordcloud = WordCloud(mask=yt_mask, background_color="white" , color_func=lambda *args, **kwargs: "red").generate(textR)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

### Pensez-vous mieux connaitres vos habitudes concernant le visionnage de vidéos?

La plateforme Youtube propose des choix de vidéos,enchaine les vidéos,... Pensez vous que les résultats suivant serais les même si il n'y avait pas toute cette incitation mais simplement la recherche?

In [ ]:
print('La première vidéo que vous avez regardé date du: {0}, et était: \"{1}\". Pour la revoir allez sur ce lien:  \"{2}\" . \n'.format(dateHwF,titleHwF,lienHwF))

print('Dans les données que nous récupérons, il arrive que les données les plus ancienne de l\'historique ne stocke pas le titre mais seulement le lien. Mais si vous le suivez vous pourrez la redécouvrir.')
# il y a un moyen d'éliminer ces lignes la mais cela peut enlevr bcp de ligne pour un fichier de 13000 peut en faire perdre 2000
print('\nLa dernière vidéo que vous avez regardé date du: {0}, et était: \"{1}\".  Pour la revoir allez sur ce lien:  \"{2}\" .\n'.format(dateHwL,titleHwL,lienHwL))

### Avez vous une idée de l'évolution du nombre de vidéo regardée?
Pensez vous que la courbe d'évolution soit la même que pour les recherches?

In [ ]:
fig6=summary_watchA.plot(kind='bar', subplots=False).set_title("Nombre de vidéos regardées en fonction des années")

Comme cela est peut etre le cas avec vos données on se rend compte que la profondeur temporelle de notre historique de vue est plus faible que pour l'historique de recherche. Cela viens de la très forte différence (multiplication par 10 en moyenne) entre le nombre de recherche et le nombre de vidéos regardées.
A cause de cela les représentations graphiques ci dessus sont moins pertinentes.

### Quelles sont vos habitudes sur une semaine?

In [ ]:
fig5=summary_watch.plot(kind='line', subplots=False)
fig5.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
fig5.set_title("Nombre de vidéos regardées en fonction \n des années et des jours de la semaine")
fig7=summary_watchJ.plot(kind='line', subplots=False).set_title("Nombre de vidéos regardées en fonction des jours de la semaine")
plt.show()

### Connaissez-vous vos vidéos favorites?

In [ ]:
print('Les vidéos que vous avez le plus regardé sont \"{0[0]}\": {1[0]} fois, \"{0[1]}\": {1[1]} fois, \"{0[2]}\": {1[2]} fois, \"{0[3]}\": {1[3]} fois, \"{0[4]}\": {1[4]} fois. '.format(Xw,Yw))

Cela correspond-il à ce que vous imaginiez? 

La question pour l'analyse de vidéo YT n'est pas de trouver des choses confidentielles mais de voir des habitudes ou des préférences. Nous avons tous une part de secret que nous n'avons pas envie de partager.
Seriez vous prêt à laisser l'accès à d'autre personnes à ces données? Cela est enfait déjà le cas, youtube premièrement pour vous proposer des choix de vidéo, des pub. Google également ont accès et utilise ces données, mais également d'autres services.

Les données présentées sur vos données ne sont qu'une partie de qu'il est possible de voir, pour aller plus loin il est possible d'utiliser une API. Une API youtube (https://developers.google.com/youtube/v3/getting-started) est disponible et permet d'obtenir encore plus d'information, par exemple de classer les vidéos par groupes (musique, cuisine, sport, série,...). Son implémention est difficile à mettre en place (notament la récupération de la bonne API), et cela n'a pas été possible dans le délais de ce projet. Mais Des outils ont déjà été dévellopés par d'autres, donc si vous voulez aller encore plus loin sur vos données vous pouvez aller tester:
*https://github.com/Jessime/youtube_history
*https://towardsdatascience.com/explore-your-activity-on-youtube-with-r-how-to-analyze-and-visualize-your-personal-data-history-b171aca632bc
En raison de problèmes d'API nous n'avons pas réussi (malgrés les bonnes explications) à utiliser les projet proposés.
Attention les nom de fichier et de dossier sont différents en français et anglais!